In [ ]:
import pandas as pd

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

## READ TRAIN DADA

In [ ]:
df_train = pd.read_csv("train.csv",encoding='latin_1')

In [ ]:
df_train.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df_train.tail(10)

,id,title,author,text,label
19941,20790,Nikki Haley Blasts U.N. Human Rights Office fo...,Adam Shaw,U. S Ambassador to the United Nations Nikki Ha...,0
19942,20791,Lawyer Who Kept Hillary Campaign Chief Out of ...,Daniel Greenfield,Lawyer Who Kept Hillary Campaign Chief Out of ...,1
19943,20792,"Jakarta Bombing Kills Three Police Officers, L...",John Hayward,Two suicide bombers attacked a bus station in ...,0
19944,20793,Idiot Who Destroyed Trump Hollywood Star Gets ...,Robert Rich,Share This \nAlthough the vandal who thought i...,1
19945,20794,Trump: Putin âVery Smartâ to Not Retaliate...,Lee Stranahan,Donald Trump took to Twitter Friday to praise ...,0
19946,20795,Rapper T.I.: Trump a âPoster Child For White...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
19947,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
19948,20797,Macyâs Is Said to Receive Takeover Approach ...,Michael J. de la Merced and Rachel Abrams,The Macyâs of today grew from the union of s...,0
19949,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
19950,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [ ]:
df_train.shape

(19951, 5)

##  EDA

In [ ]:
#import matplotlib.pyplot as plt


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19951 entries, 0 to 19950
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      19951 non-null  int64 
 1   title   19422 non-null  object
 2   author  18079 non-null  object
 3   text    19914 non-null  object
 4   label   19951 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 779.5+ KB


In [ ]:
df_train.describe()

,id,label
count,19951.000000,19951.000000
mean,10806.595309,0.499674
std,5790.205228,0.500012
min,0.000000,0.000000
25%,5836.500000,0.000000
50%,10824.000000,0.000000
75%,15811.500000,1.000000
max,20799.000000,1.000000


In [ ]:
df_train.isnull().sum()

id           0
title      529
author    1872
text        37
label        0
dtype: int64

In [ ]:
#df_train=df_train.dropna()

In [ ]:
df_train=df_train.dropna()

In [ ]:
df_train.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

## split data in x and y

In [ ]:
y = df_train['label']

In [ ]:
y.head(5)

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [ ]:
x = df_train.drop('label',axis=1)

In [ ]:
x.head(5)

,id,title,author,text
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
x['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

#@

In [ ]:
messages=x.copy()

In [ ]:
messages.head(5)

,id,title,author,text
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
messages.reset_index(inplace=True)

In [ ]:
messages.head(5)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Preprocessing

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size=5000

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

## Embedding Representati

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2587 4633 2320]
 [   0    0    0 ... 1030 1556 4050]
 [   0    0    0 ... 1923 3050 2329]
 ...
 [   0    0    0 ... 3057 1469 4362]
 [   0    0    0 ... 4202 2254 3848]
 [   0    0    0 ... 1450   60 2943]]


## Create new dataframe

In [ ]:
for i in range(10):
  print(embedded_docs[i])

[   0    0    0    0    0    0    0    0    0    0 4847 2985  160 1307
 4654 3806 1138 2587 4633 2320]
[   0    0    0    0    0    0    0    0    0    0    0    0    0 3944
 4921   43  485 1030 1556 4050]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0 1914 1923 3050 2329]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
 3804 3532 4131 3008 3891 2365]
[   0    0    0    0    0    0    0    0    0    0 4482 1030 1787 1705
 2924 3739 1030 2156 2705  368]
[   0    0    0    0    0  456  105 2083 2492 1548 4245 3011 4274 3069
 3603 4502 3598 3831  638 4050]
[   0    0    0    0    0    0    0    0    0 2764  848  888 3902 1026
 2322   49  897 3057 1469 4362]
[   0    0    0    0    0    0    0    0    0 1126 3101 3942 3725 3639
 2416 4245 1426 3057 1469 4362]
[   0    0    0    0    0    0    0    0    0    0 3551 2871 3976 3618
 1314 1659 3262 2362 4245 2039]
[   0    0    0    0    0    0    0    0    0    0    0    0 4202 1001
 1

In [ ]:
len(embedded_docs)

17550

In [ ]:
a=[]
for i in range(20):
  s=(str(i))
  a.append(('a'+s))

In [ ]:
a

['a0',
 'a1',
 'a2',
 'a3',
 'a4',
 'a5',
 'a6',
 'a7',
 'a8',
 'a9',
 'a10',
 'a11',
 'a12',
 'a13',
 'a14',
 'a15',
 'a16',
 'a17',
 'a18',
 'a19']

In [ ]:
df_2 = pd.DataFrame(columns=a)

In [ ]:
df_2

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19


In [ ]:
for i in range(len(embedded_docs)):
  df_2.loc[i] = embedded_docs[i]

In [ ]:
df_2.head(5)

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19
0,0,0,0,0,0,0,0,0,0,0,4847,2985,160,1307,4654,3806,1138,2587,4633,2320
1,0,0,0,0,0,0,0,0,0,0,0,0,0,3944,4921,43,485,1030,1556,4050
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1914,1923,3050,2329
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3804,3532,4131,3008,3891,2365
4,0,0,0,0,0,0,0,0,0,0,4482,1030,1787,1705,2924,3739,1030,2156,2705,368


In [ ]:
df_2.tail(20)

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19
17530,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,477,4791,1820,2620
17531,0,0,0,0,0,0,0,0,0,2266,1030,4807,2705,3813,266,171,2105,3057,1469,4362
17532,0,0,0,0,0,0,0,0,0,0,0,0,4362,2234,4374,4894,3120,1740,4772,3486
17533,0,0,0,0,0,0,0,4044,3420,1961,1242,4006,35,2921,2454,4791,4921,888,4345,4245
17534,0,0,0,0,0,0,0,0,0,2686,1848,4331,1009,2989,3896,3714,3629,3057,1469,4362
17535,0,0,0,0,0,0,0,0,0,0,0,4772,2620,3919,943,4201,4245,797,1735,1702
17536,0,0,0,0,0,0,0,0,0,0,2420,1086,780,2863,2657,2005,1612,2595,2203,2717
17537,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4840,921,974,4245,1503,4050
17538,0,0,0,0,0,0,0,0,0,0,2807,2598,2186,937,625,1877,4385,2378,3082,638
17539,0,0,0,0,0,0,0,0,0,0,4898,716,472,2543,1721,2664,4345,4245,888,4050


In [ ]:
#df_2.reset_index(inplace=True)

In [ ]:
#df_2.head(5)

In [ ]:
df_2.shape

(17550, 20)

In [ ]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17550 entries, 0 to 17549
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   a0      17550 non-null  int32
 1   a1      17550 non-null  int32
 2   a2      17550 non-null  int32
 3   a3      17550 non-null  int32
 4   a4      17550 non-null  int32
 5   a5      17550 non-null  int32
 6   a6      17550 non-null  int32
 7   a7      17550 non-null  int32
 8   a8      17550 non-null  int32
 9   a9      17550 non-null  int32
 10  a10     17550 non-null  int32
 11  a11     17550 non-null  int32
 12  a12     17550 non-null  int32
 13  a13     17550 non-null  int32
 14  a14     17550 non-null  int32
 15  a15     17550 non-null  int32
 16  a16     17550 non-null  int32
 17  a17     17550 non-null  int32
 18  a18     17550 non-null  int32
 19  a19     17550 non-null  int32
dtypes: int32(20)
memory usage: 1.5 MB


In [ ]:
df_2.describe()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19
count,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000,17550.000000
mean,6.519772,6.893447,11.447123,14.068205,19.022279,37.167350,71.370427,157.197151,335.195840,661.244843,1075.816011,1446.169459,1805.123362,2081.233732,2292.005128,2404.251624,2500.401766,2710.425926,2163.687407,3331.399202
std,149.467741,151.845909,204.582843,222.951764,248.860521,360.970484,498.412903,723.346426,1011.577732,1339.746642,1579.967771,1670.138630,1688.557669,1641.151173,1581.821093,1517.648494,1466.828830,1215.981933,1282.485703,1393.453910
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,338.250000,877.250000,1155.250000,1273.250000,1994.000000,1469.000000,2378.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,569.000000,1572.000000,2057.000000,2331.500000,2438.000000,2558.000000,3057.000000,1469.000000,4050.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,43.000000,2105.000000,2896.000000,3327.750000,3543.500000,3680.000000,3708.250000,3742.000000,3135.000000,3084.000000,4362.000000
max,4928.000000,4921.000000,4978.000000,4937.000000,4921.000000,4939.000000,4995.000000,4999.000000,4996.000000,4996.000000,4998.000000,4998.000000,4998.000000,4998.000000,4996.000000,4996.000000,4998.000000,4993.000000,4998.000000,4997.000000


In [ ]:
df_2.isnull().sum()

a0     0
a1     0
a2     0
a3     0
a4     0
a5     0
a6     0
a7     0
a8     0
a9     0
a10    0
a11    0
a12    0
a13    0
a14    0
a15    0
a16    0
a17    0
a18    0
a19    0
dtype: int64

In [ ]:
x=df_2

In [ ]:
x

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19
0,0,0,0,0,0,0,0,0,0,0,4847,2985,160,1307,4654,3806,1138,2587,4633,2320
1,0,0,0,0,0,0,0,0,0,0,0,0,0,3944,4921,43,485,1030,1556,4050
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1914,1923,3050,2329
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3804,3532,4131,3008,3891,2365
4,0,0,0,0,0,0,0,0,0,0,4482,1030,1787,1705,2924,3739,1030,2156,2705,368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17545,0,0,0,0,0,0,0,0,0,0,0,0,0,0,875,4245,2686,3648,538,3887
17546,0,0,0,0,0,0,0,0,0,0,1803,60,4763,3548,1182,3906,3641,3057,1469,4362
17547,0,0,0,0,0,0,0,0,0,0,2132,4720,462,223,4411,3740,4426,3057,1469,4362
17548,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,3639,358,4202,2254,3848


##MinMaxScaler

In [ ]:
#from sklearn.preprocessing import MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x)
X=scaler.transform(x)

## StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
scaler.fit(x)
x_1 = scaler.transform(x)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X1_train,X1_test,Y1_train,Y1_test = train_test_split(x_1,y,test_size=0.25,random_state=45)

In [ ]:
## Classification

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=75)

## LogisticRegression_min

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lg = LogisticRegression()
lg.fit(x_train,y_train)
y_pred = lg.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(accuracy_score(y_test,y_pred))

0.7857549857549857


In [ ]:
l_s = LogisticRegression()
l_s.fit(X1_train,Y1_train)
y_pred_1=l_s.predict(X1_test)

In [ ]:
print(accuracy_score(y_pred_1,Y1_test))

0.795123062898815


## K_Nearest

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
kn = KNeighborsClassifier(n_neighbors=15)
kn.fit(x_train,y_train)
yn_pred = kn.predict(x_test)

In [ ]:
print(accuracy_score(y_test,yn_pred))

0.7851851851851852


## GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb = GaussianNB()
nb.fit(x_train,y_train)
y_pred = nb.predict(x_test)

In [ ]:
print(accuracy_score(y_pred,y_test))

0.5802469135802469


## SVM

In [ ]:
"""
s=['linear', 'poly', 'rbf', 'sigmoid']
a_s=[]
for i in range(0,len(s)):
    df1=svm.SVC(kernel=s[i])
    df1.fit(x_train,y_train)
    y_pred=df1.predict(x_test)
    A_S=accuracy_score(y_test,y_pred)
    a_s.append(A_S)
"""

"\ns=['linear', 'poly', 'rbf', 'sigmoid']\na_s=[]\nfor i in range(0,len(s)):\n    df1=svm.SVC(kernel=s[i])\n    df1.fit(x_train,y_train)\n    y_pred=df1.predict(x_test)\n    A_S=accuracy_score(y_test,y_pred)\n    a_s.append(A_S)\n"

In [ ]:
"""
for i in range(len(a_s)):
    print("{0} = {1}".format(s[i],a_s[i]))
"""

'\nfor i in range(len(a_s)):\n    print("{0} = {1}".format(s[i],a_s[i]))\n'

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

## LSTM

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(17550, (17550,))

In [ ]:
y = df_train['label']

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((17550, 20), (17550,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=7,batch_size=30)

Epoch 1/7
392/392 [==============================] - 11s 27ms/step - loss: 9.9372e-06 - accuracy: 1.0000 - val_loss: 0.9372 - val_accuracy: 0.9071
Epoch 2/7
392/392 [==============================] - 11s 29ms/step - loss: 7.6455e-06 - accuracy: 1.0000 - val_loss: 0.9560 - val_accuracy: 0.9068
Epoch 3/7
392/392 [==============================] - 11s 29ms/step - loss: 6.0252e-06 - accuracy: 1.0000 - val_loss: 0.9709 - val_accuracy: 0.9076
Epoch 4/7
392/392 [==============================] - 12s 30ms/step - loss: 4.5875e-06 - accuracy: 1.0000 - val_loss: 0.9900 - val_accuracy: 0.9073
Epoch 5/7
392/392 [==============================] - 10s 25ms/step - loss: 3.8109e-06 - accuracy: 1.0000 - val_loss: 1.0097 - val_accuracy: 0.9073
Epoch 6/7
392/392 [==============================] - 11s 28ms/step - loss: 2.9682e-06 - accuracy: 1.0000 - val_loss: 1.0286 - val_accuracy: 0.9076
Epoch 7/7
392/392 [==============================] - 11s 29ms/step - loss: 2.3156e-06 - accuracy: 1.0000 - val_loss: 1

In [ ]:

"""from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
"""

"from tensorflow.keras.layers import Dropout\n## Creating model\nembedding_vector_features=40\nmodel=Sequential()\nmodel.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))\nmodel.add(Dropout(0.3))\nmodel.add(LSTM(100))\nmodel.add(Dropout(0.3))\nmodel.add(Dense(1,activation='sigmoid'))\nmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])\n"

In [ ]:
"""y_pred=model.predict_classes(X_test)"""

AttributeError: ignored

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=-1)

181/181 [==============================] - 1s 8ms/step


In [ ]:
y_pred.shape

(5792,)

In [ ]:
y_test.shape

(5792,)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3267,    0],
       [2525,    0]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5640538674033149